In [ ]:
import pandas as pd
import numpy as np

def calculate_optimal_pnl(df, heat_rate=10, mw=250, min_run_hours=3, min_down_hours=6):
    # Calculate potential PnL for each hour
    df['potential_pnl'] = (df['power_price'] - (df['gas_price'] * heat_rate)) * mw
    
    n = len(df)
    # Initialize the columns for decisions and PnL
    df['run_decision'] = False
    df['pnl'] = 0.0
    
    # Use dynamic programming to calculate maximum possible PnL
    dp = [0] * (n + 1)  # DP array to store maximum PnL at each point
    path = [-1] * n  # To store decisions
    
    for i in range(n - 1, -1, -1):
        # Calculate PnL if plant stays off at this hour
        dp[i] = dp[i + 1]
        path[i] = i + 1  # Move to next hour without running
        
        # Calculate PnL if plant runs starting this hour
        run_profit = 0
        for j in range(i, min(i + min_run_hours, n)):
            run_profit += df.iloc[j]['potential_pnl']
            if j - i + 1 >= min_run_hours:  # Check if meets minimum running time
                if j + min_down_hours < n:  # Ensure enough time for minimum down time
                    if dp[i] < run_profit + dp[j + min_down_hours + 1]:
                        dp[i] = run_profit + dp[j + min_down_hours + 1]
                        path[i] = j + min_down_hours + 1  # Record decision path
                else:  # No need for down time consideration
                    if dp[i] < run_profit:
                        dp[i] = run_profit
                        path[i] = n  # Indicates end of decision path
                        
    # Reconstruct decisions from path
    i = 0
    while i < n:
        if path[i] == i + 1:  # Indicates staying off
            df.at[df.index[i], 'pnl'] = 0
        else:  # Indicates running
            for j in range(i, path[i] - min_down_hours if path[i] != n else n):
                df.at[df.index[j], 'pnl'] = df.iloc[j]['potential_pnl']
                df.at[df.index[j], 'run_decision'] = True
            i = path[i] - 1  # Skip to end of running period
        i += 1
    
    return df

# Example DataFrame for demonstration
data = {
    'time': pd.date_range(start='2022-01-01', periods=24, freq='H'),
    'power_price': np.random.uniform(50, 150, 24),  # Simulated power prices
    'gas_price': np.random.uniform(3, 10, 24),      # Simulated gas prices
}
df = pd.DataFrame(data)
df.set_index('time', inplace=True)

# Call the function
optimized_df = calculate_optimal_pnl(df)

# Display the optimized DataFrame
print(optimized_df[['power_price', 'gas_price', 'potential_pnl', 'run_decision', 'pnl']])
